In [1]:
import numpy as np
import pandas as pd
import datetime as dt


In [2]:
df= pd.read_csv("Online_Retail_Try4.csv", encoding = 'unicode_escape')

#Display all results in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.head(5)
df.shape
df.describe()
df.Product.unique()
df.Product.value_counts()
df.isnull().any() # False means no missing values
df.info()  #categorical variable (object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceID    13 non-null     int64         
 1   Product      13 non-null     object        
 2   Quantity     13 non-null     int64         
 3   InvoiceDate  13 non-null     datetime64[ns]
 4   UnitPrice    13 non-null     float64       
 5   CustomerID   13 non-null     int64         
 6   Total        13 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 856.0+ bytes


In [3]:
df.sort_values(by=['CustomerID','InvoiceDate'], inplace=True)

In [4]:
df['same_invoice'] = df['InvoiceID'] == df['InvoiceID'].shift(-1)

In [142]:
products = ["A", "B", "C"]
table = {}
table_pair = []

for cus in set(df['CustomerID']):
    lead_time = []
    products_pair = []
    for i in products :
        for j in products :
            df1 = df[df["CustomerID"]==cus][df["Product"] == i]
            df2 = df[df["CustomerID"]==cus][df["Product"] == j]
            if(len(df1)>0 and len(df2)>0):
                merged_df = pd.merge(df1, df2, on="CustomerID", suffixes=("_1"+i, "_2"+j))
                merged_df['lead_time'] = merged_df.apply(lambda x: x["InvoiceDate_1"+i]- x["InvoiceDate_2"+j],axis=1).dt.days
                merged_df = merged_df[merged_df["lead_time"] > 0]
                average_lead_time = merged_df["lead_time"].mean()
                lead_time.append(average_lead_time)
                products_pair.append(j + " to " +i)
                    
            else:
                lead_time.append(None)
                products_pair.append(j + " to " +i)
                    
    table_pair.append(products_pair)       
    table[cus] =  lead_time


table_final = pd.DataFrame.from_dict(table).T
table_final.columns =  table_pair[0]
table_final['CustomerID'] = table_final.index

<ipython-input-142-0023f4f24785>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["CustomerID"]==cus][df["Product"] == i]
<ipython-input-142-0023f4f24785>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df[df["CustomerID"]==cus][df["Product"] == j]


In [143]:
table_final

,A to A,B to A,C to A,A to B,B to B,C to B,A to C,B to C,C to C,CustomerID
1,8.0,7.666667,7.666667,4.5,8.0,8.0,4.5,8.0,8.0,1
2,NaN,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
